# Глава 5 Работа с категориальными данными

# Кодирование номинальных категориальных признаков

In [ ]:
#дан признак с номинальными классами, который не имеет внутренней упорядоченности(напр. яблоко, груша, банан)

In [1]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [2]:
#создать признак
feature = np.array([
    ["Texas"],
    ["California"],
    ["Texas"],
    ["Delaware"],
    ["Texas"]
])

In [6]:
#создать кодировщик одного активного состояния
one_hot = LabelBinarizer()

#преобразовать признак в кодировку с одним активным состоянием
one_hot_transformer = one_hot.fit_transform(feature)
one_hot_transformer

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [7]:
#взглянуть на классы признака
one_hot.classes_

array(['California', 'Delaware', 'Texas'], dtype='<U10')

In [8]:
#если требуется обратить кодирование с одним активным состоянием, то можно применить метод inverse_transform

In [9]:
#обратить кодирование с одним активным состоянием
one_hot.inverse_transform(one_hot_transformer)

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

In [ ]:
#для преобразования признака в кодировку с одним активным состоянием можно использовать библиотеку pandas

In [10]:
import pandas as pd 
#создать фиктивные переменные из признака
pd.get_dummies(feature[:,0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


In [ ]:
#обработка ситуации, когда в каждом наблюдении перечисляется несколько классов

In [11]:
#создать мультиклассовый признак
multiclass_feature = np.array([
    ["Texas", "Florida"],
    ["California", "Alabama"],
    ["Texas", "Florida"],
    ["Delaware", "Florida"],
    ["Texas", "Alabama"]
])

In [12]:
#создать мультиклассовый кодировщик, преобразующий признак
#в кодировку с одним активным состоянием
one_hot_multiclass = MultiLabelBinarizer()

In [14]:
#кодировать мультиклассовый признак в кодировку с одним активным состоянием
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [15]:
#взглянуть на классы
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

# Кодирование порядковых категориальных признаков

In [ ]:
#Дан порядковый категориальный признак(высокий, средний, низкий).
#Выполнить его кодировку.

In [2]:
import pandas as pd
dataframe = pd.DataFrame({"оценка": ["низкая", "низкая", "средняя", "средняя", "высокая"]})
#создать словарь преобразования шкалы
scale_mapper = {"низкая": 1,
               "средняя": 2,
               "высокая": 3}
print(dataframe)

    оценка
0   низкая
1   низкая
2  средняя
3  средняя
4  высокая


In [3]:
#заменить значения признаков, значениями словаря
dataframe["оценка"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: оценка, dtype: int64

In [5]:
dataframe = pd.DataFrame({"оценка": ["низкая", "низкая", "средняя", "средняя", "высокая", "чуть больше средней"]})
scale_mapper = {"низкая": 1,
               "средняя": 2,
                "чуть больше средней": 2.1,
               "высокая": 3}
dataframe["оценка"].replace(scale_mapper)

0    1.0
1    1.0
2    2.0
3    2.0
4    3.0
5    2.1
Name: оценка, dtype: float64

# Кодирование словарей признаков

In [ ]:
#дан словарь и требуется его конвертировать в матрицу признаков

In [11]:
from sklearn.feature_extraction import DictVectorizer
#создать словарь
data_dict = [{"красный": 2, "синий": 4},
            {"красный": 2, "синий": 4},
            {"красный": 2, "желтый": 4},
            {"красный": 2, "желтый": 4}]
#создать векторизатор словаря
dictvectorizer = DictVectorizer(sparse=False)

In [12]:
#конвертировать словарь в матрицу признаков
features = dictvectorizer.fit_transform(data_dict)
print(features)

[[0. 2. 4.]
 [0. 2. 4.]
 [4. 2. 0.]
 [4. 2. 0.]]


In [14]:
#получить имена признаков
feature_names = dictvectorizer.get_feature_names()
feature_names

['желтый', 'красный', 'синий']

In [15]:
import pandas as pd
pd.DataFrame(features, columns=feature_names)

,желтый,красный,синий
0,0.0,2.0,4.0
1,0.0,2.0,4.0
2,4.0,2.0,0.0
3,4.0,2.0,0.0


# Импутация пропущенных значений классов

In [ ]:
#дан категориальный признак, содержащий пропущенные значения, 
#которые требуется заменить предсказанными значениями
#используем KNN

In [16]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [18]:
#создать матрицу признаков с категориальным признаком
X = np.array([[0, 2.10, 1.45],
             [1, 1.18, 1.33],
             [0, 1.22, 1.27],
             [1, -0.21, -1.19]])
#создать матрицу признаков с отсутствующими значениями в категориальном признаке
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                      [np.nan, -0.67, -0.22]])
#обучение
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:, 1:], X[:, 0])

In [20]:
#предсказать класс пропущенных значений
imputed_values = trained_model.predict(X_with_nan[:, 1:])

In [22]:
#соединить столбец предсказанного класса с другими признаками
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:, 1:]))
#соединить две матрицы признаков
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

In [ ]:
#альтернативное решение - заполнение пропущенных значений наиболее частыми значениями признаков

In [23]:
from sklearn.preprocessing import Imputer

In [24]:
#соединить две матрицы признаков
X_complete = np.vstack((X_with_nan, X))
imputer = Imputer(strategy='most_frequent', axis=0)
imputer.fit_transform(X_complete)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

# Работа с несбалансированными классами

In [ ]:
#дан вектор целей с очень несбалансированными классами

In [13]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

In [14]:
iris = load_iris()

In [15]:
#создать матрицу признаков
features = iris.data

In [16]:
#создать вектор целей
target = iris.target

In [17]:
#удалить первые 40 наблюдений
features = features[40:,:]
target = target[40:]
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [18]:
#создать бинарный вектор целей, указывающий, является ли класс нулевым
target = np.where((target == 0), 0, 1)
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
#создать веса
weights = {0: .9, 1: 0.1}

In [23]:
#создать классификатор на основе случайного леса с весами
RandomForestClassifier(class_weight=weights)
RandomForestClassifier(bootstrap=True, class_weight={0: .9, 1: 0.1}, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight={0: 0.9, 1: 0.1},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [ ]:
#либо передать аргумент balanced, который автоматически создает веса, 
#обратно пропорциональные частотам классов
RandomForestClassifier(class_weight="balanced")

In [24]:
RandomForestClassifier(bootstrap=True, class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [26]:
#Индекса наблюдений каждого класса
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
print('i_class0', i_class0)
print('i_class1', i_class1)

i_class0 [0 1 2 3 4 5 6 7 8 9]
i_class1 [ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109]


In [27]:
np.where(target == 0)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),)

In [28]:
np.where(target == 1)

(array([ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
         23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
         36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
         49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
         62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
         75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int64),)

In [ ]:
#количество наблюдений в каждом классе

In [32]:
n_class0 = len(i_class0)
n_class0

10

In [33]:
n_class1 = len(i_class1)
n_class1

100

In [35]:
#для каждого наблюдения класса 0 сделать случайную выборку из класса 1 без возврата
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)


In [36]:
#соединить вектор целей класса 0 с вектором целей пониженного класса 1
np.hstack((target[i_class0], target[i_class1_downsampled]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [37]:
#соединить матрицу признаков класса 0 с матрицей признаков пониженного класса 1
np.vstack((features[i_class0, :], features[i_class1_downsampled, :]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

In [38]:
#другой вариант - повысить миноритарный класс
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

In [39]:
#соединить повышенный вектор целей класса 0 с вектором целей класса 1
np.concatenate((target[i_class0_upsampled],target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [40]:
#соединить повышенную матрицу признаков класса 0 с матрицей признака класса 1
np.vstack((features[i_class0_upsampled, :], features[i_class1, :]))[0:5]

array([[5. , 3.5, 1.6, 0.6],
       [4.6, 3.2, 1.4, 0.2],
       [5.1, 3.8, 1.6, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [5.1, 3.8, 1.9, 0.4]])